In [4]:
import mido as md
import numpy as np
from collections import defaultdict
import sklearn as sk

In [1]:
def generate_transition_probs(filepath):
    poops = []
    song = md.MidiFile(filepath)
    
    for track in song.tracks:
        poop = np.zeros((127, 127))
        prevnote = None
        n = np.zeros(127)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on' and x['velocity'] != 0:
                note = x['note']
                if prevnote is not None:
                    poop[prevnote][note] += 1
                    n[prevnote] += 1
                prevnote = note

        for i in range(127):
            if (n[i] != 0):
                poop[i] /= n[i]
        poops.append(poop)
    return poops


In [2]:
def transpose(filepath, offset):
    result = md.MidiFile()
    
    song = md.MidiFile(filepath)
    for track in song.tracks:
        newtrack = md.MidiTrack()
        result.tracks.append(newtrack)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on':
                note = x['note']
                note += offset
                newmessage = md.Message('note_on', note=note, velocity=x['velocity'], time=x['time'])
                newtrack.append(newmessage)
            else:
                newtrack.append(msg)
    result.save("midis/1.mid")

In [3]:
def findKeySignature(song):
    for i, track in enumerate(song.tracks):
        for msg in track:
            if msg.type == 'key_signature':
                return msg.key
            
def findTransposeOffset(filepath):
    song = md.MidiFile(filepath)
    keys = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']
    keySig = findKeySignature(song)
    print(keySig)
    for i in range(len(keys)):
        if keySig in keys[i]:
            offset = i
            break
    return offset

def removeZeroNotes(matrix):
    for j in (len(matrix)):
        i = len(matrix) - j - 1
        if not row.any(): #this means it's all zero
            np.delete(matrix, i, 0)
            np.delete(matrix, i, 1)
    return matrix
    
    
def findStationaryDist(matrix):
    try:
        w, v = np.linalg.eig(matrix.T)
    except LinAlgError as e:
        return None
    if 1 in w:
        return v[list(w).index(1)]
    else:
        return None

In [5]:

song = md.MidiFile('./midis/chopinblack.mid')
pMatrices = defaultdict(list)
for i, track in enumerate(song.tracks):
    pMatrix = np.zeros((127, 127))
    prevNote = None
    for msg in track:
        print(msg)
        if msg.type == 'note_on' and msg.velocity:
            if prevNote:
                pMatrix[prevNote][msg.note] += 1
            prevNote = msg.note
    if pMatrix.any():
        pMatrices[i] = pMatrix

for track, pMatrix in pMatrices.items():
    pMatrices[track] = sk.preprocessing.normalize(pMatrix, axis=1, norm='l1')


<meta message track_name name='Etüde Opus 10 No. 5' time=0>
<meta message track_name name='Schwarze-Tasten-Etüde' time=0>
<meta message copyright text='Copyright © 1999 von Bernd Krüger.' time=0>
<meta message text text='Chopin' time=0>
<meta message text text='Vivace' time=0>
<meta message text text='Fertiggestellt am 30.3.99\n' time=0>
<meta message text text='Update am 1.4.99\n' time=0>
<meta message text text='Update am 21.4.99\n' time=0>
<meta message text text='Normierung: 23.12.2002\n' time=0>
<meta message text text='Update am 4.3.2014\n' time=0>
<meta message text text='Dauer: 1:31\n' time=0>
<meta message time_signature numerator=2 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='Gb' time=0>
<meta message set_tempo tempo=517241 time=0>
<meta message marker text='Vivace' time=0>
<meta message set_tempo tempo=530973 time=3360>
<meta message set_tempo tempo=517241 time=480>
<meta message set_tempo tempo=538117 time=2640>
<m

note_on channel=0 note=80 velocity=85 time=0
note_on channel=0 note=80 velocity=0 time=80
note_on channel=0 note=92 velocity=90 time=0
note_on channel=0 note=92 velocity=0 time=80
note_on channel=0 note=82 velocity=94 time=0
note_on channel=0 note=82 velocity=0 time=80
note_on channel=0 note=94 velocity=86 time=0
note_on channel=0 note=94 velocity=0 time=80
note_on channel=0 note=87 velocity=83 time=0
note_on channel=0 note=87 velocity=0 time=80
note_on channel=0 note=94 velocity=97 time=0
note_on channel=0 note=94 velocity=0 time=80
note_on channel=0 note=80 velocity=80 time=0
note_on channel=0 note=80 velocity=0 time=80
note_on channel=0 note=92 velocity=88 time=0
note_on channel=0 note=92 velocity=0 time=80
note_on channel=0 note=85 velocity=98 time=0
note_on channel=0 note=85 velocity=0 time=80
note_on channel=0 note=94 velocity=90 time=0
note_on channel=0 note=94 velocity=0 time=80
note_on channel=0 note=80 velocity=87 time=0
note_on channel=0 note=80 velocity=0 time=80
note_on ch

AttributeError: module 'sklearn' has no attribute 'preprocessing'